In [1]:
import pandas as pd 
import numpy as np 
import pywt
import os
import pickle
import re 
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit,LeaveOneOut,KFold
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

In [2]:
sys.path.append('.')
sys.path.append('..')

In [3]:
from Fourier import fourier_feature_fusion as eff

In [4]:
from Wavelet import wavelet_feature_fusion as wff 

In [5]:
destination_folder = "FusedDataSlidingWindowNew"

In [6]:
user_no=2
user_folder='../../../User2/'
last_index_even=41 # For even user numbers
last_index_odd=40  # For odd user numbers
increment=2        # Number for the increment of the user numbers

In [7]:
def getLabel(label):
    new_label=[]
    for i in range(len(label)):
        if(label[i]>4.5):
            new_label.append(1)
        else:
            new_label.append(0)
    return new_label

In [8]:
# clean data
def clean_data(dataset, target):
    del_rows = []
    for i in range(len(dataset)):
        if(np.isnan(dataset[i]).sum() > 0):
            del_rows.append(i)
            
    dataset = np.delete(dataset, del_rows, axis=0)
    target = np.delete(target, del_rows, axis=0)
    
    return dataset,target

In [9]:
window_size = 512
step_size = 512
User_nums = ['26']

In [10]:
arousal=[]
valence=[]
dominance=[]
liking=[]

In [11]:
def get_full_features(user_no, window_size,step_size):
    
    ar_dict = {}
    val_dict = {}
    dom_dict = {}
    lik_dict = {}
    
    user_folder='../../../User'+ str(user_no) + '/'
    
    user_csv= os.listdir(user_folder)
    
    user_csv = user_csv[1:]
    
    user_labels_folder = user_folder + 'Label/'
    
    user_labels=os.listdir(user_labels_folder)
    
    all_data=[]
    all_labels=[]
    
    for csv in user_csv:
        all_data.append(csv)
    
    for lab in user_labels:
        all_labels.append(lab)
        
    final_features=[]
    final_labels=[]
        
    for i in range(0,len(all_data)):
        
        data = pd.read_csv(user_folder + all_data[i])
        cols = data.columns
        cols = cols[:14]
        labels=pd.read_csv(user_labels_folder + all_labels[i])
        labels=labels.values[0][:4]
        
        start = 0;
        while start + window_size < data.shape[0]:
            
            temp_array = []
            temp_data = [] 
            
            for i in cols:
                X = data[i][start : start + window_size]
                features_fourier = eff.power_spectrum(X)
                features_wavelet = wff.wavelet_energy(X)
                features_fourier = np.array(features_fourier).ravel()
                features_wavelet = np.array(features_wavelet).ravel()
                temp_array.append(np.concatenate((features_fourier,features_wavelet),axis=0).ravel())
                  
            temp_array=np.array(temp_array).ravel()
            final_features.append(temp_array) 
            final_labels.append(labels)
            start = start + step_size
                
    final_features = np.array(final_features)
    final_labels = np.array(final_labels)
#     print(final_features.shape)
#     print(final_labels.shape)
    
    arousal_dataset=final_features
    valence_dataset=final_features
    dominance_dataset=final_features
    liking_dataset=final_features
    
    arousal_labels=final_labels[:,0]
    valence_labels=final_labels[:,1]
    dominance_labels=final_labels[:,2]
    liking_labels=final_labels[:,3]
    
    
    arousal_dataset,arousal_labels = clean_data(arousal_dataset,arousal_labels)
    valence_dataset,valence_labels = clean_data(valence_dataset,valence_labels)
    dominance_dataset ,dominance_labels = clean_data(dominance_dataset,dominance_labels)
    liking_dataset,liking_labels = clean_data(liking_dataset,liking_labels)
    
    
    arousal_labels = getLabel(arousal_labels)
    valence_labels = getLabel(valence_labels)
    dominance_labels = getLabel(dominance_labels)
    liking_labels = getLabel(liking_labels)
    
    ar_count=len(set(arousal_labels))
    if ar_count==1:
        ar_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
    
    else:
        sm1 = SMOTE()
        arousal_dataset,arousal_labels = sm1.fit_resample(arousal_dataset,arousal_labels)
        x_a_train,x_a_test,y_a_train,y_a_test = train_test_split(arousal_dataset,arousal_labels,test_size=0.3,random_state=42)
        ar_model = SVC(kernel="rbf",C = 10)
        ar_model.fit(x_a_train,y_a_train)
        pred_a = ar_model.predict(x_a_test)
        ar_acc=accuracy_score(pred_a,y_a_test)
        ar_f1=f1_score(y_a_test, pred_a, average='macro')
        
        ar_dict[user_no]=['Accuracy: '+str(ar_acc),'F1-Score: '+str(ar_f1)]
        arousal.append(ar_dict)
        
        
    val_count=len(set(valence_labels))
    if val_count==1:
        val_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
        
    else:
        sm2 = SMOTE()
        valence_dataset,valence_labels = sm2.fit_resample(valence_dataset,valence_labels)
        x_v_train,x_v_test,y_v_train,y_v_test = train_test_split(valence_dataset,valence_labels,test_size=0.3,random_state=42)
        val_model = SVC(kernel="rbf",C = 10)
        val_model.fit(x_v_train,y_v_train)
        pred_v = val_model.predict(x_v_test)
        val_acc=accuracy_score(pred_v,y_v_test)
        val_f1=f1_score(y_v_test, pred_v, average='macro')
        
        val_dict[user_no]=['Accuracy: '+str(val_acc),'F1-Score: '+str(val_f1)]
        valence.append(val_dict)
        
        
    dom_count=len(set(dominance_labels))
    if dom_count==1:
        dom_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]
    
    else:
        sm3 = SMOTE()
        dominance_dataset,dominance_labels = sm3.fit_resample(dominance_dataset,dominance_labels)
        x_d_train,x_d_test,y_d_train,y_d_test = train_test_split(dominance_dataset,dominance_labels,test_size = 0.3,random_state = 42)
        dom_model = SVC(kernel="rbf",C = 10)
        dom_model.fit(x_d_train,y_d_train)
        pred_d = dom_model.predict(x_d_test)
        dom_acc=accuracy_score(pred_d,y_d_test)
        dom_f1=f1_score(y_d_test, pred_d, average='macro')
        
        dom_dict[user_no]=['Accuracy: '+str(dom_acc),'F1-Score: '+str(dom_f1)]
        dominance.append(dom_dict)
        
    
    lik_count=len(set(liking_labels))
    if lik_count==1:
        lik_dict[user_no]=['Accuracy: '+str(100),'F1-Score: '+str(1)]

    else:
        sm4 = SMOTE()
        liking_dataset,liking_labels = sm4.fit_resample(liking_dataset,liking_labels)
        x_l_train,x_l_test,y_l_train,y_l_test = train_test_split(liking_dataset,liking_labels,test_size = 0.3,random_state = 42)
        lik_model = SVC(kernel="rbf",C = 10)
        lik_model.fit(x_l_train,y_l_train)
        pred_l = lik_model.predict(x_l_test)
        lik_acc=accuracy_score(pred_l,y_l_test)
        lik_f1=f1_score(y_l_test, pred_l, average='macro')
 
        lik_dict[user_no]=['Accuracy: '+str(lik_acc),'F1-Score: '+str(lik_f1)]
        liking.append(lik_dict)
    
    
    print('User:' + str(user_no)+ " "+"Done")

In [12]:
for i in range(user_no,last_index_even,2):
    get_full_features(i,window_size,step_size)

User:2 Done
User:4 Done
User:6 Done
User:8 Done
User:10 Done
User:12 Done
User:14 Done
User:16 Done
User:18 Done
User:20 Done
User:22 Done
User:24 Done
User:26 Done
User:28 Done
User:30 Done
User:32 Done
User:34 Done
User:36 Done
User:38 Done
User:40 Done


In [13]:
arousal

[{2: ['Accuracy: 0.8918558077436582', 'F1-Score: 0.8916820359185219']},
 {4: ['Accuracy: 0.9656319290465631', 'F1-Score: 0.9656315488664508']},
 {6: ['Accuracy: 0.9776536312849162', 'F1-Score: 0.9776502552116109']},
 {8: ['Accuracy: 0.9468085106382979', 'F1-Score: 0.9461996336996337']},
 {10: ['Accuracy: 0.9593984962406015', 'F1-Score: 0.9592791583225417']},
 {12: ['Accuracy: 0.9661214953271028', 'F1-Score: 0.9661177498276816']},
 {14: ['Accuracy: 0.908284023668639', 'F1-Score: 0.908271176944357']},
 {16: ['Accuracy: 0.9278074866310161', 'F1-Score: 0.9277992277992277']},
 {18: ['Accuracy: 0.972936400541272', 'F1-Score: 0.9729339720769422']},
 {20: ['Accuracy: 0.9082969432314411', 'F1-Score: 0.9082689556509299']},
 {22: ['Accuracy: 0.992867332382311', 'F1-Score: 0.9928644863142679']},
 {24: ['Accuracy: 0.9439252336448598', 'F1-Score: 0.9438811188811189']},
 {26: ['Accuracy: 0.932806324110672', 'F1-Score: 0.9327596359789536']},
 {32: ['Accuracy: 0.945273631840796', 'F1-Score: 0.945251949

In [14]:
valence

[{2: ['Accuracy: 0.8746031746031746', 'F1-Score: 0.8745320476460579']},
 {4: ['Accuracy: 0.8653061224489796', 'F1-Score: 0.8645751327448451']},
 {6: ['Accuracy: 0.9327102803738317', 'F1-Score: 0.9325384593829686']},
 {8: ['Accuracy: 0.6818181818181818', 'F1-Score: 0.6805244378059911']},
 {10: ['Accuracy: 0.8773946360153256', 'F1-Score: 0.8773298083278255']},
 {12: ['Accuracy: 0.90625', 'F1-Score: 0.9053587960118301']},
 {14: ['Accuracy: 0.7971602434077079', 'F1-Score: 0.7970191040843215']},
 {16: ['Accuracy: 0.8908045977011494', 'F1-Score: 0.890688660702805']},
 {18: ['Accuracy: 0.9494584837545126', 'F1-Score: 0.9494162916585143']},
 {20: ['Accuracy: 0.8556701030927835', 'F1-Score: 0.8549145299145299']},
 {22: ['Accuracy: 0.8530701754385965', 'F1-Score: 0.8530355433480687']},
 {24: ['Accuracy: 0.8823529411764706', 'F1-Score: 0.8822510822510823']},
 {26: ['Accuracy: 0.7041666666666667', 'F1-Score: 0.7035439029906574']},
 {28: ['Accuracy: 0.8103448275862069', 'F1-Score: 0.810288432946773

In [15]:
dominance

[{2: ['Accuracy: 0.9087893864013267', 'F1-Score: 0.9087080391428217']},
 {4: ['Accuracy: 0.8422090729783037', 'F1-Score: 0.8420314690761801']},
 {6: ['Accuracy: 0.9683615819209039', 'F1-Score: 0.9683401984934126']},
 {8: ['Accuracy: 0.6754385964912281', 'F1-Score: 0.6752136752136753']},
 {10: ['Accuracy: 0.9848993288590604', 'F1-Score: 0.9848921410404821']},
 {12: ['Accuracy: 0.9412955465587044', 'F1-Score: 0.9411680157038607']},
 {14: ['Accuracy: 0.9863713798977853', 'F1-Score: 0.9863624747345677']},
 {16: ['Accuracy: 0.9244264507422402', 'F1-Score: 0.9244197059552084']},
 {18: ['Accuracy: 0.9833119383825417', 'F1-Score: 0.9832762222460759']},
 {20: ['Accuracy: 0.8148893360160966', 'F1-Score: 0.8144197675173712']},
 {22: ['Accuracy: 0.9971590909090909', 'F1-Score: 0.9971590679805977']},
 {24: ['Accuracy: 0.7183098591549296', 'F1-Score: 0.7178060413354531']},
 {26: ['Accuracy: 0.8722139673105498', 'F1-Score: 0.872213685177335']},
 {28: ['Accuracy: 0.8406593406593407', 'F1-Score: 0.8398

In [16]:
liking

[{2: ['Accuracy: 0.9540757749712974', 'F1-Score: 0.9540248086566376']},
 {4: ['Accuracy: 0.9617224880382775', 'F1-Score: 0.961720516269353']},
 {6: ['Accuracy: 0.9743888242142026', 'F1-Score: 0.9743860124695038']},
 {8: ['Accuracy: 0.8188405797101449', 'F1-Score: 0.8136645962732918']},
 {10: ['Accuracy: 0.9898648648648649', 'F1-Score: 0.9898648520118751']},
 {12: ['Accuracy: 0.9350393700787402', 'F1-Score: 0.9348553442011386']},
 {14: ['Accuracy: 0.9264705882352942', 'F1-Score: 0.9261614371590396']},
 {16: ['Accuracy: 0.9547581903276131', 'F1-Score: 0.9547048345869975']},
 {18: ['Accuracy: 0.9588477366255144', 'F1-Score: 0.9588303063160748']},
 {20: ['Accuracy: 0.9099236641221374', 'F1-Score: 0.9094373138420654']},
 {22: ['Accuracy: 0.9307535641547862', 'F1-Score: 0.930399906610633']},
 {24: ['Accuracy: 0.8888888888888888', 'F1-Score: 0.888888888888889']},
 {26: ['Accuracy: 0.9765100671140939', 'F1-Score: 0.9765100377235731']},
 {28: ['Accuracy: 0.8113207547169812', 'F1-Score: 0.810050

In [17]:
final_dict={}

In [18]:
final_dict['Arousal']=arousal
final_dict['Valence']=valence
final_dict['Dominance']=dominance
final_dict['Liking']=liking

In [19]:
df = pd.DataFrame(list())
df.to_csv('even_svc_ff_sl.csv')

In [20]:
with open('even_svc_ff_sl.csv', 'w') as f:
    for key in final_dict.keys():
        f.write("%s,%s\n"%(key,final_dict[key]))